In [ ]:
import numpy as np
import pandas as pd
import plotnine as gg
import pandas_datareader as pdr
import twitter

pd.options.display.max_rows = 1000

In [ ]:
df_clean = pd.read_csv('../data/engineered_factset_campaign.csv', parse_dates=['campaign_announcement_date'])

In [ ]:
df_clean.groupby(['activist_id', 'activist_name']).campaign_id.count().sort_values(ascending=False).to_clipboard()

In [ ]:
df_clean.groupby(['company_id', 'company_name']).campaign_id.count().sort_values(ascending=False).to_clipboard()

In [ ]:
df_clean[df_clean.activist_name == "Carl Celian Icahn"].head(5)

In [ ]:
df_clean.iloc[0].to_frame('value')

In [ ]:
df_clean.dtypes.to_frame('type')

In [ ]:
len(df_clean)

In [ ]:
df_clean[[c for c in df_clean.columns.tolist() if 'price_to_earnings' in c]].describe()

In [ ]:
df_clean[[c for c in df_clean.columns.tolist() if 'return' in c]].isnull().sum()

In [ ]:
df_clean.shape

In [ ]:
df_clean.hist(figsize=(50, 50));

# Twitter

In [ ]:
api = twitter.Api(
    consumer_key="2h4XsRFKDcCcPUocDkG56NziZ",
    consumer_secret="C6uYqGrwmVgBaQ6GxjKfL7XrRD9G7gNHd7W7GvfuLyfWWsIP3e",
    access_token_key="1198652056241877004-BgWT67aYNYXlePsQzYEFtkcvMwxK6D",
    access_token_secret="czxwtySpTHLZh6xh2YHwb42QTOywkANubHYCGYXCighOw",
    sleep_on_rate_limit=True
)

In [ ]:
def tweets_to_dataframe(tweets):
    
    df = pd.DataFrame.from_dict([t._json for t in timeline])
    df = df.assign(created_at=lambda df: pd.to_datetime(df.created_at))
    return df

In [ ]:
timeline = api.GetUserTimeline(screen_name="Carl_C_Icahn", count=5, include_rts=False, exclude_replies=True)
tweets_to_dataframe(timeline)

In [ ]:
search_string = "Exxon Mobil Corporation ".replace(' ', '%20')
since = "2010-09-15"
until = "2011-07-20"
query = f"""q={search_string}since%3A{since}&until%3A{until}&count=5"""
results = api.GetSearch(raw_query=query, count=15)
df_twitter = pd.DataFrame.from_dict([t._json for t in results])
df_twitter = df_twitter.assign(created_at=lambda df: pd.to_datetime(df.created_at))

In [ ]:
dict_tweets = {}

def clean_company_name(company_name):
    return (
        company_name
        .replace('Co.', '')
        .replace('Inc.', '')
        .replace('(NRO)', '')
        .replace('.', '')
        .replace(',', '')
        .replace('/', '')
        .replace(' and ', '')
        .replace(' & ', '')
        .replace('\'s', '')
    )

for campaign_id, gb in df_clean.groupby('campaign_id'):
    
    company_id = gb.company_id.iloc[0]
    company_name = gb.company_name.iloc[0]
    
    if company_name and type(company_name) == str:
    
        campaign_date = gb.campaign_announcement_date.iloc[0]
        start_date = campaign_date + pd.offsets.DateOffset(-60)
        end_date = campaign_date + pd.offsets.DateOffset(-60)

        print(f'getting tweets for {clean_company_name(company_name)}')

        search_string = (clean_company_name(company_name) + ' ').replace(' ', '%20')
        since = start_date.strftime('%Y-%m-%d')
        until = end_date.strftime('%Y-%m-%d')

        query = f"""q={search_string}since%3A{since}&until%3A{until}&count=100"""

        try:
            results = api.GetSearch(raw_query=query, count=15)
            if results:
                df_result = pd.DataFrame.from_dict([t._json for t in results])
            else:
                print(f'found no tweets for {company_name}')
                df_result = None
        except Exception:
            print(f'request failed for {company_name}')
            df_result = None
            
        if df_result is not None:
            df_result = df_result.assign(company_id=company_id).assign(company_name=company_name)

        dict_tweets[campaign_id] = df_result
    
df_tweets = pd.concat(dict_tweets, names=['campaign_id'], sort=False).reset_index()
df_tweet_count = df_tweets.groupby(['campaign_id', 'company_id', 'company_name']).text.count().to_frame('tweet_count').reset_index()

In [ ]:
df_tweets = pd.concat(dict_tweets, names=['campaign_id'], sort=False).reset_index()
df_tweet_count = df_tweets.groupby(['campaign_id', 'company_id', 'company_name']).text.count().to_frame('tweet_count').reset_index()

In [ ]:
df_tweets.head()

In [ ]:
df_tweet_count

In [ ]:
df_tweets.to_csv('../data/twitter_tweets.csv')

In [ ]:
df_tweet_count.to_csv('../data/twitter_count.csv')

# Pricing

In [ ]:
df_pricing = pd.read_csv('../data/clean_factset_pricing.csv')

In [ ]:
len(df_pricing)

In [ ]:
len(df_pricing.company_id.unique())

# Campaigns

Keyed by `(campaign_id, activist_id, company_id)`.

In [ ]:
df_clean.campaign_id.nunique()

In [ ]:
df_campaign = (
    df_clean
    .groupby('campaign_id')
    .last()
    .reset_index()
)

In [ ]:
df_campaign.head(5)

In [ ]:
df_campaign.groupby('campaign_objective_primary').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_campaign.groupby('value_demand').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_campaign.groupby('governance_demand').campaign_id.count().sort_values(ascending=False).to_frame('count')

In [ ]:
df_campaign[[c for c in df_campaign if 'return' in c]].describe()

# Tactics

Keyed by `(campaign_id, activist_id, company_id, activist_campaign_tactic)`.

In [ ]:
def clean_column_name(column_name):
    return (
        column_name
        .strip()
        .lower()
        .replace(' ', '_')
        .replace('-', '_')
        .replace('\n', '_')
        .replace('(', '')
        .replace(')', '')
        .replace('%', 'pecent')
        .replace('/', '_or_')
        .replace('?', '')
        .replace('__', '_')
        .replace('_dates', '_date')
        .replace('_announcements', '_announcement')
        .replace('pre_date', 'pre')
        .replace('post_date', 'post')
        .replace('18_months_pre', 'pre_18m')
        .replace('1_year_pre', 'pre_12m')
        .replace('6_months_pre', 'pre_6m')
        .replace('90_days_pre', 'pre_3m')
        .replace('18_months_post', 'post_18m')
        .replace('1_year_post', 'post_12m')
        .replace('6_months_post', 'post_6m')
        .replace('90_days_post', 'post_3m')        
    )

df_tactic = (
    df_clean
    .groupby('campaign_id')
    [
        'activist_id',
        'company_id',
        'activist_campaign_tactic'
    ]
    .last()
    .reset_index()
    .assign(activist_campaign_tactic=lambda df: df.activist_campaign_tactic.fillna('No or Unknown'))
    .assign(activist_campaign_tactic=lambda df: df.activist_campaign_tactic.str.split(', '))
    .explode('activist_campaign_tactic')
    .assign(activist_campaign_tactic_indicator=1)
)

In [ ]:
df_tactic.head()

In [ ]:
df_tactic.groupby('activist_campaign_tactic').campaign_id.count().sort_values(ascending=False).to_frame('count').to_clipboard()

# Activists

Keyed by `(activist_id)`.

In [ ]:
df_clean.activist_id.nunique()

In [ ]:
df_activist = (
    df_clean
    .groupby('activist_id')
    ['activist_name', 'activist_group']
    .last()
    .reset_index()
)

df_activist.head(5)

In [ ]:
(
    pd.merge(
        df_activist,
        df_campaign.groupby('activist_id').campaign_id.count().to_frame('campaign_count'),
        how='left',
        on=['activist_id']
    )
    .sort_values(by='campaign_count', ascending=False)
    .head(10)
)

# Targets

Keyed by `(company_id)`.

In [ ]:
df_clean.company_id.nunique()

In [ ]:
df_company = (
    df_clean
    .groupby('company_id')
    [
        'company_name',
        'sector',
        'current_entity_status',
        'current_entity_detail'
    ]
    .last()
    .reset_index()
)

In [ ]:
df_company.head(10)